In [1]:
# included modules necessary for the code for this notebook.
import xarray as xr
import os
import numpy as np
import gsw
from netCDF4 import Dataset
import glob
import netCDF4 as nc
import numpy.ma as ma
import matplotlib.pyplot as plt
import cmocean
from numpy import savetxt

This is a snippet (Jan 2012) of the kinetic energy flux code to show how to apply the coarse graining approach, fairly hard-coded.
Breaks down the components of the flux calculations as outlined in equation (1) in the manuscript

1) load in the HF radar data from whereever it is saved
2) extract the latitude, longitude, and velocity components
3) Apply the filter function where radius is the filtered scale - for the purposes of the study, 7 km was used, but any filtered scale could be applied
4) create the derivative grid space, that is different from the latitude and longitudes pulled from the HF radar
5) create the stencil for the appropriate filtered scale (7 km shown below)
6) apply the coarse graining function and filtered scale/stencil to the different components of the kinetic energy flux computations

In [104]:
# 'a' is the data load file - load in your you data from wherever you stored it
a = xr.open_mfdataset('/export/data1/jbenjami/Data/SMODE/HF_Radar/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best_2012_01*.nc', parallel=True)
latitudes = a['lat']
latitude = latitudes[153:264].values
longitudes = a['lon']
longitude = longitudes[449:593].values
uvel = a['u']
Janu_mean = uvel[:,153:264,449:593].values
vvel = a['v']
Janv_mean = vvel[:,153:264,449:593].values



In [105]:
# This function takes the velocity data on the lat/lon grid and applies the top hat filter of the coarse graining method at the appropriate radius, where the radius is the filtering scale
def observ(velocity, lat, lon, radius):
    ny, nx = velocity.shape
    
    start_row = max(lat - radius, 0)
    # print(start_row)
    end_row = min(lat + radius + 1, ny)
    # print(end_row)
    
    start_col = max(lon - radius, 0)
    # print(start_col)
    end_col = min(lon + radius + 1, nx)
    # print(end_col)
    
    return velocity[start_row:end_row, start_col:end_col]

Setting up the grid initially and only needed once for all spatial plots

In [ ]:
# determining dy - constant over all longitude lines
long = [longitude[25], longitude[25]]
lat1 = [latitude[50], latitude[51]]
lat2 = [latitude[54], latitude[55]]
distance1 = gsw.distance(long, lat1)
distance2 = gsw.distance(long, lat2)

dy = (distance1+distance2)/2

[1999.56086808]
[1999.1366931]


array([1999.34878059])

In [ ]:
# determining dx - not constant over all latitude lines
dxs = []
for i in range(0, len(latitude)):
    long = [longitude[0], longitude[1]]
    lat = [latitude[i], latitude[i]]
    distance = gsw.distance(long, lat)
    dxs.append(distance)
    
dx = np.squeeze(dxs)

In [ ]:
"""
These are the arrays of latitude and longitude on which the velocity derivatives will lie
The derivatives aren't on the generic latitude/longitude grid provided because in computing derivatives for 
vorticity, divergence, strain, etc, the grid gets shifted
"""
dlat = (latitude[1]-latitude[0])/2
lat = latitude + dlat
dlon = np.abs((longitude[0]-longitude[1])/2)
lon = longitude + dlon

Creating the stencil that will be called - this shows the filter stencil of values that fall within the filtered scale and anything beyond will not be applied to the calculation in that iteration

In [109]:
# l = 7 km
np.sqrt((3*np.nanmean(dx))**2+(1*dy)**2)
stencil7 = np.ones((7,7))
stencil7[0,0] = np.nan
stencil7[0,6] = np.nan
stencil7[0,5] = np.nan
stencil7[1,6] = np.nan
stencil7[0,1] = np.nan
stencil7[1,0] = np.nan
stencil7[6,0] = np.nan
stencil7[5,0] = np.nan
stencil7[6,1] = np.nan
stencil7[6,6] = np.nan
stencil7[6,5] = np.nan
stencil7[5,6] = np.nan
stencil7

array([[nan, nan,  1.,  1.,  1., nan, nan],
       [nan,  1.,  1.,  1.,  1.,  1., nan],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [nan,  1.,  1.,  1.,  1.,  1., nan],
       [nan, nan,  1.,  1.,  1., nan, nan]])

filtered velocites - 2 day average to eliminate tidal effects

In [1]:
au_mean = np.full_like(Janu_mean, np.nan)
av_mean = np.full_like(Janu_mean, np.nan)

for p in range(0, len(Janu_mean)):
    au_mean[p] = np.nanmean(Janu_mean[p:(p+48),:,:], axis=0)
    av_mean[p] = np.nanmean(Janv_mean[p:(p+48),:,:], axis=0)

### Calculating the kinetic energy flux and each component as delineated in equation (1) in the manuscript

I personally saved the calculations as I went along (not shown here) - you can save whatever data you need

In [2]:
# first for the u-velocity
filteredu = np.full(((744,111,144)), np.nan)
for p in range(0,744): 
    u = au_mean[p,:,:]
    ufiltered = np.full((111,144), np.nan)

    velocity = u
    radius = 3

    for i in range(radius,len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            result = observ(velocity, i, j, radius)*stencil7
            # print(result)
            average = (np.nanmean(result))
            # print(average)
            ufiltered[i,j] = average
    filteredu[p] = ufiltered

In [124]:
# filtered 2 km/SCA kinetic energy flux u-velocity derivatives
dudx5 = np.full(((744,(len(latitude)-1),(len(longitude)-1))), np.nan) # one less on each dimension than the filteredu
dudy5 = np.full_like(dudx5, np.nan)
for p in range(0, 744):
    ufiltered = filteredu[p,:,:]
    dufildx5 = np.full((len(latitude)-1, len(longitude)-1),np.nan)
    dufildy5 = np.full_like(dufildx5, np.nan)
    for j in range(radius, (len(latitude)-radius-1)):
        for i in range(radius, (len(longitude)-radius-1)):
            dufildx5[j,i] = (((ufiltered[j,(i+1)]-ufiltered[j,i])/dx[j]) + ((ufiltered[(j+1),(i+1)]-ufiltered[(j+1),i])/dx[(j+1)]))/2
            dufildy5[j,i] = ((ufiltered[(j+1),i]-ufiltered[j,i])/dy + (ufiltered[(j+1),(i+1)]-ufiltered[j, (i+1)])/dy)/2
    dudx5[p]=dufildx5
    dudy5[p]=dufildy5

In [3]:
# now for the v-velocity
filteredv = np.full_like(filteredu, np.nan)
for p in range(0, 744):
    v = av_mean[p,:,:]
    vfiltered = np.full_like(ufiltered, np.nan)

    velocity = v
    radius = 3

    for i in range(radius, len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            result = observ(velocity, i, j, radius)*stencil7
    #         print(result.shape)
            average = (np.nanmean(result))
            vfiltered[i,j] = average
    filteredv[p] = vfiltered

In [128]:
# filtered 2 km/SCA kinetic energy flux v-velocity derivatives
dvdx5 = np.full(((744,(len(latitude)-1),(len(longitude)-1))), np.nan) # one less on each dimension than the filteredu
dvdy5 = np.full_like(dvdx5, np.nan)
for p in range(0, 744):
    vfiltered = filteredv[p,:,:]
    dvfildx5 = np.full((len(latitude)-1, len(longitude)-1),np.nan)
    dvfildy5 = np.full_like(dvfildx5, np.nan)
    for j in range(radius, (len(latitude)-(radius-1))):
        for i in range(radius, (len(longitude)-(radius-1))):
            dvfildx5[j,i] = (((vfiltered[j,(i+1)]-vfiltered[j,i])/dx[j]) + ((vfiltered[(j+1),(i+1)]-vfiltered[(j+1),i])/dx[(j+1)]))/2
            dvfildy5[j,i] = ((vfiltered[(j+1),i]-vfiltered[j,i])/dy + (vfiltered[(j+1),(i+1)]-vfiltered[j, (i+1)])/dy)/2
    dvdx5[p]=dvfildx5
    dvdy5[p]=dvfildy5

In [130]:
"""
Now taking velocity squared
first u-velocity squared
"""
filteredusqrd = np.full_like(filteredu, np.nan)
filteredvsqrd = np.full_like(filteredu, np.nan)
filtereduv = np.full_like(filteredu, np.nan)
for p in range(0,744):
    u = au_mean[p,:,:]
    v = av_mean[p,:,:]
    usqrdfiltered = np.full_like(ufiltered, np.nan)

    velocity = u**2
    radius = 3

    for i in range(radius,len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            result = observ(velocity, i, j, radius)*stencil7
    #         print(result.shape)
            average = (np.nanmean(result))
            usqrdfiltered[i,j] = average
    filteredusqrd[p] = usqrdfiltered

# now v-velocity squared
    vsqrdfiltered = np.full_like(ufiltered, np.nan)

    velocity = v**2
    radius = 3

    for i in range(radius,len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            result = observ(velocity, i, j, radius)*stencil7
    #         print(result.shape)
            average = (np.nanmean(result))
            vsqrdfiltered[i,j] = average
    filteredvsqrd[p] = vsqrdfiltered
    
# now u-v velocity
    uvfiltered = np.full_like(ufiltered, np.nan)

    velocity = u*v
    radius = 3

    for i in range(radius,len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            result = observ(velocity, i, j, radius)*stencil7
    #         print(result.shape)
            average = (np.nanmean(result))
            uvfiltered[i,j] = average
    filtereduv[p] = uvfiltered
    

/tmp/ipykernel_26107/2760086185.py:18: RuntimeWarning: Mean of empty slice
  average = (np.nanmean(result))
/tmp/ipykernel_26107/2760086185.py:32: RuntimeWarning: Mean of empty slice
  average = (np.nanmean(result))
/tmp/ipykernel_26107/2760086185.py:46: RuntimeWarning: Mean of empty slice
  average = (np.nanmean(result))


In [132]:
# now calculating tau - so first calculate tau on the longitude/latitude grid then average to place on the lon/lat grid
uutau = np.full_like(filteredu, np.nan)
tauuu5 = np.full(((744, (len(latitude)-1), (len(longitude)-1))), np.nan)
for p in range(0, 744):
    uutau[p] = filteredusqrd[p,:,:] - filteredu[p,:,:]*filteredu[p,:,:]

for p in range(0, 744):
    tau = uutau[p,:,:]
    Tau = np.full_like(dufildx5, np.nan)
    for i in range(radius, len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            Tau[i,j] = (tau[i,j]+tau[i,j+1]+tau[i+1,j+1]+tau[i+1,j])/4
    tauuu5[p] = Tau
    
vvtau = np.full_like(filteredu, np.nan)
tauvv5 = np.full(((744, (len(latitude)-1), (len(longitude)-1))), np.nan)
for p in range(0, 744):
    vvtau[p] = filteredvsqrd[p,:,:] - filteredv[p,:,:]*filteredv[p,:,:]

for p in range(0, 744):
    tau = vvtau[p,:,:]
    Tau = np.full_like(dufildx5, np.nan)
    for i in range(radius, len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            Tau[i,j] = (tau[i,j]+tau[i,j+1]+tau[i+1,j+1]+tau[i+1,j])/4
    tauvv5[p] = Tau
    
uvtau = np.full_like(filteredu, np.nan)
tauuv5 = np.full(((744, (len(latitude)-1), (len(longitude)-1))), np.nan)
for p in range(0, 744):
    uvtau[p] = filtereduv[p,:,:] - filteredu[p,:,:]*filteredv[p,:,:]

for p in range(0, 744):
    tau = uvtau[p,:,:]
    Tau = np.full_like(dufildx5, np.nan)
    for i in range(radius, len(latitude)-radius):
        for j in range(radius, len(longitude)-radius):
            Tau[i,j] = (tau[i,j]+tau[i,j+1]+tau[i+1,j+1]+tau[i+1,j])/4
    tauuv5[p] = Tau

In [134]:
# finally, take each component, and calculate the flux
pi = -(tauuv5*(dudy5+dvdx5)+tauuu5*dudx5+tauvv5*dvdy5)